# Geração de Estudantes Sintéticos para Simulação SINKT

Este notebook implementa a **Etapa 2** do pipeline SINKT: criação de estudantes sintéticos baseados nos perfis cognitivos gerados anteriormente.

## Objetivo
Gerar 100 estudantes sintéticos com variação individual controlada, distribuídos entre os diferentes perfis cognitivos.

## Entrada
- `data/output/notebooks/geracao_perfis/profiles.json`: Perfis cognitivos gerados no notebook anterior

## Saída
- `data/output/notebooks/geracao_estudantes/students.json`: Arquivo JSON contendo todos os estudantes sintéticos
- `data/output/notebooks/geracao_estudantes/students.md`: Documentação em Markdown dos estudantes gerados

## Importação de Bibliotecas

Carregamento das bibliotecas necessárias para manipulação de dados, geração aleatória e validação.

In [1]:
import json
import os
import numpy as np
from datetime import datetime
from typing import Dict, List, Any
from collections import Counter

## Configurações de Geração

Define os parâmetros para geração dos estudantes sintéticos:
- **NUM_STUDENTS**: Número total de estudantes a gerar
- **SEED**: Seed para reprodutibilidade
- **INDIVIDUAL_VARIATION**: Percentual de variação individual (15%)
- **PROFILE_DISTRIBUTION**: Distribuição dos estudantes entre os perfis

In [2]:
# Configurações
NUM_STUDENTS = 100
SEED = 42
INDIVIDUAL_VARIATION = 0.15

# Distribuição dos perfis (soma = 1.0)
PROFILE_DISTRIBUTION = {
    "balanced": 0.30,
    "quick_learner": 0.20,
    "careful": 0.20,
    "struggling": 0.10,
    "logical": 0.10,
    "intuitive": 0.10
}

# Configurar seed
np.random.seed(SEED)

print(f"⚙️  Configurações:")
print(f"  - Número de estudantes: {NUM_STUDENTS}")
print(f"  - Seed: {SEED}")
print(f"  - Variação individual: ±{INDIVIDUAL_VARIATION*100:.0f}%")
print(f"\n📊 Distribuição esperada por perfil:")
for profile_id, pct in PROFILE_DISTRIBUTION.items():
    expected = int(NUM_STUDENTS * pct)
    print(f"  - {profile_id}: {pct*100:.0f}% (~{expected} estudantes)")

⚙️  Configurações:
  - Número de estudantes: 100
  - Seed: 42
  - Variação individual: ±15%

📊 Distribuição esperada por perfil:
  - balanced: 30% (~30 estudantes)
  - quick_learner: 20% (~20 estudantes)
  - careful: 20% (~20 estudantes)
  - struggling: 10% (~10 estudantes)
  - logical: 10% (~10 estudantes)
  - intuitive: 10% (~10 estudantes)


## Carregamento dos Perfis Cognitivos

Carrega os perfis gerados no notebook anterior a partir do arquivo JSON.

In [3]:
# Carregar perfis
profiles_file = 'data/output/notebooks/geracao_perfis/profiles.json'

if not os.path.exists(profiles_file):
    raise FileNotFoundError(
        f"❌ Arquivo de perfis não encontrado: {profiles_file}\n"
        "Execute primeiro o notebook '01_geracao_perfis.ipynb'"
    )

with open(profiles_file, 'r', encoding='utf-8') as f:
    profiles_data = json.load(f)

profiles = profiles_data['profiles']

print(f"✅ Perfis carregados: {len(profiles)} perfis")
print(f"\n📋 Perfis disponíveis:")
for profile_id, profile in profiles.items():
    print(f"  - {profile['nome']} ({profile_id})")

✅ Perfis carregados: 6 perfis

📋 Perfis disponíveis:
  - Estudante Equilibrado (balanced)
  - Aprendiz Rápido (quick_learner)
  - Estudante Cuidadoso (careful)
  - Estudante com Dificuldades (struggling)
  - Pensador Lógico (logical)
  - Estudante Intuitivo (intuitive)


## Função de Geração de Estudantes

Implementa a lógica de geração de estudantes com variação individual controlada.
Cada estudante herda os parâmetros do perfil base com uma variação aleatória de ±15%.

In [4]:
def apply_individual_variation(value: float, variation: float = INDIVIDUAL_VARIATION) -> float:
    """Aplica variação individual a um parâmetro, mantendo no range [0, 1]."""
    noise = np.random.uniform(-variation, variation)
    new_value = value * (1 + noise)
    return np.clip(new_value, 0.0, 1.0)

def generate_student(student_id: int, profile_id: str, profile: Dict[str, Any]) -> Dict[str, Any]:
    """Gera um estudante sintético baseado em um perfil."""
    numeric_params = ['mastery_init_level', 'learn_rate', 'slip', 'guess', 
                      'logic_skill', 'reading_skill', 'tech_familiarity',
                      'memory_capacity', 'learning_consistency']
    
    student = {
        'id': f"student_{student_id:04d}",
        'profile_id': profile_id,
        'profile_name': profile['nome']
    }
    
    # Aplicar variação individual aos parâmetros numéricos
    for param in numeric_params:
        if param in profile:
            student[param] = round(apply_individual_variation(profile[param]), 4)
    
    return student

print("✅ Função de geração definida")

✅ Função de geração definida


## Geração dos Estudantes Sintéticos

Gera o conjunto completo de estudantes distribuídos conforme a configuração definida.

In [5]:
students = []
student_counter = 0

# Gerar estudantes para cada perfil
for profile_id, proportion in PROFILE_DISTRIBUTION.items():
    if profile_id not in profiles:
        print(f"⚠️  Aviso: Perfil '{profile_id}' não encontrado, pulando...")
        continue
    
    num_students_for_profile = int(NUM_STUDENTS * proportion)
    profile = profiles[profile_id]
    
    for _ in range(num_students_for_profile):
        student = generate_student(student_counter, profile_id, profile)
        students.append(student)
        student_counter += 1

# Ajustar para atingir exatamente NUM_STUDENTS (caso haja arredondamento)
while len(students) < NUM_STUDENTS:
    # Adicionar estudantes extras do perfil mais comum
    most_common_profile = max(PROFILE_DISTRIBUTION, key=PROFILE_DISTRIBUTION.get)
    profile = profiles[most_common_profile]
    student = generate_student(student_counter, most_common_profile, profile)
    students.append(student)
    student_counter += 1

print(f"✅ Gerados {len(students)} estudantes sintéticos")

# Mostrar distribuição real
actual_distribution = Counter([s['profile_id'] for s in students])
print(f"\n📊 Distribuição real por perfil:")
for profile_id, count in sorted(actual_distribution.items()):
    percentage = (count / len(students)) * 100
    print(f"  - {profile_id}: {count} estudantes ({percentage:.1f}%)")

✅ Gerados 100 estudantes sintéticos

📊 Distribuição real por perfil:
  - balanced: 30 estudantes (30.0%)
  - careful: 20 estudantes (20.0%)
  - intuitive: 10 estudantes (10.0%)
  - logical: 10 estudantes (10.0%)
  - quick_learner: 20 estudantes (20.0%)
  - struggling: 10 estudantes (10.0%)


## Validação dos Estudantes Gerados

Verifica se todos os estudantes possuem parâmetros válidos e IDs únicos.

In [6]:
def validate_students(students: List[Dict[str, Any]]) -> List[str]:
    """Valida os estudantes sintéticos."""
    errors = []
    
    required_fields = ['id', 'profile_id', 'profile_name', 'mastery_init_level', 
                       'learn_rate', 'slip', 'guess', 'logic_skill', 
                       'reading_skill', 'tech_familiarity', 'memory_capacity',
                       'learning_consistency']
    
    numeric_fields = ['mastery_init_level', 'learn_rate', 'slip', 'guess', 
                      'logic_skill', 'reading_skill', 'tech_familiarity',
                      'memory_capacity', 'learning_consistency']
    
    student_ids = set()
    
    for i, student in enumerate(students):
        # Verifica campos obrigatórios
        for field in required_fields:
            if field not in student:
                errors.append(f"Estudante {i}: Campo '{field}' ausente")
        
        # Verifica ID único
        if 'id' in student:
            if student['id'] in student_ids:
                errors.append(f"Estudante {i}: ID '{student['id']}' duplicado")
            student_ids.add(student['id'])
        
        # Verifica ranges dos parâmetros numéricos
        for field in numeric_fields:
            if field in student:
                value = student[field]
                if not isinstance(value, (int, float)):
                    errors.append(f"Estudante {student.get('id', i)}: '{field}' deve ser numérico")
                elif value < 0 or value > 1:
                    errors.append(f"Estudante {student.get('id', i)}: '{field}' = {value} fora do range [0, 1]")
        
        # Verifica se profile_id existe
        if 'profile_id' in student and student['profile_id'] not in profiles:
            errors.append(f"Estudante {student.get('id', i)}: profile_id '{student['profile_id']}' inválido")
    
    return errors

# Executar validação
validation_errors = validate_students(students)

if validation_errors:
    print("❌ Erros de validação encontrados:")
    for error in validation_errors[:10]:  # Mostrar apenas os primeiros 10
        print(f"  - {error}")
    if len(validation_errors) > 10:
        print(f"  ... e mais {len(validation_errors) - 10} erros")
else:
    print("✅ Todos os estudantes são válidos!")

✅ Todos os estudantes são válidos!


## Cálculo de Estatísticas dos Estudantes

Calcula estatísticas descritivas para cada parâmetro dos estudantes gerados.

In [7]:
def calculate_student_statistics(students: List[Dict[str, Any]]) -> Dict[str, Dict[str, float]]:
    """Calcula estatísticas dos parâmetros dos estudantes."""
    numeric_fields = ['mastery_init_level', 'learn_rate', 'slip', 'guess', 
                      'logic_skill', 'reading_skill', 'tech_familiarity',
                      'memory_capacity', 'learning_consistency']
    
    stats = {}
    
    for field in numeric_fields:
        values = [s[field] for s in students if field in s]
        if values:
            stats[field] = {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }
    
    return stats

# Calcular e exibir estatísticas
stats = calculate_student_statistics(students)

print("\n📊 Estatísticas dos Parâmetros dos Estudantes:\n")
for param, values in stats.items():
    print(f"  {param}:")
    print(f"    Média: {values['mean']:.3f}")
    print(f"    Desvio: {values['std']:.3f}")
    print(f"    Range: [{values['min']:.3f}, {values['max']:.3f}]")
    print()


📊 Estatísticas dos Parâmetros dos Estudantes:

  mastery_init_level:
    Média: 0.469
    Desvio: 0.094
    Range: [0.231, 0.631]

  learn_rate:
    Média: 0.041
    Desvio: 0.021
    Range: [0.013, 0.089]

  slip:
    Média: 0.147
    Desvio: 0.046
    Range: [0.068, 0.258]

  guess:
    Média: 0.136
    Desvio: 0.037
    Range: [0.071, 0.229]

  logic_skill:
    Média: 0.580
    Desvio: 0.169
    Range: [0.265, 0.962]

  reading_skill:
    Média: 0.580
    Desvio: 0.156
    Range: [0.265, 0.907]

  tech_familiarity:
    Média: 0.549
    Desvio: 0.174
    Range: [0.175, 0.911]

  memory_capacity:
    Média: 0.633
    Desvio: 0.140
    Range: [0.352, 0.911]

  learning_consistency:
    Média: 0.704
    Desvio: 0.168
    Range: [0.346, 1.000]



## Criação do Arquivo JSON de Saída

Estrutura e salva os estudantes em formato JSON com metadados completos.

In [8]:
# Criar estrutura completa com metadados
output_data = {
    "metadata": {
        "description": "Conjunto de estudantes sintéticos gerados para validação do SINKT",
        "version": "1.0.0",
        "created_at": datetime.now().isoformat(),
        "num_students": len(students),
        "generation_seed": SEED,
        "individual_variation": INDIVIDUAL_VARIATION,
        "profile_distribution": PROFILE_DISTRIBUTION
    },
    "students": students
}

# Criar diretório de saída se não existir
os.makedirs('data/output/notebooks/geracao_estudantes', exist_ok=True)

# Salvar arquivo JSON
output_file = 'data/output/notebooks/geracao_estudantes/students.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ Estudantes salvos em: {output_file}")
print(f"📦 Total de estudantes: {len(students)}")

✅ Estudantes salvos em: data/output/notebooks/geracao_estudantes/students.json
📦 Total de estudantes: 100


## Resumo da Execução

Exibe um resumo final com informações sobre os estudantes gerados e próximos passos.

In [9]:
print("\n" + "="*60)
print("🎉 GERAÇÃO DE ESTUDANTES CONCLUÍDA COM SUCESSO!")
print("="*60)
print(f"\n📁 Arquivos gerados:")
print(f"  - {output_file}")
print(f"\n📊 Resumo:")
print(f"  - Total de estudantes: {len(students)}")
print(f"  - Perfis utilizados: {len(actual_distribution)}")
print(f"  - Seed de geração: {SEED}")
print(f"\n✅ Pipeline de geração sintética completo!")
print(f"\n💡 Próximos passos sugeridos:")
print(f"  1. Integrar com modelo SINKT")
print(f"  2. Simular interações com questões")
print(f"  3. Validar com dados reais")
print("="*60)


🎉 GERAÇÃO DE ESTUDANTES CONCLUÍDA COM SUCESSO!

📁 Arquivos gerados:
  - data/output/notebooks/geracao_estudantes/students.json

📊 Resumo:
  - Total de estudantes: 100
  - Perfis utilizados: 6
  - Seed de geração: 42

✅ Pipeline de geração sintética completo!

💡 Próximos passos sugeridos:
  1. Integrar com modelo SINKT
  2. Simular interações com questões
  3. Validar com dados reais
